# Importing Libraries

In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

2023-05-06 18:23:48.882794: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-06 18:23:48.932136: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dense, Flatten
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Functions to Load Datasets

In [3]:
def load_labels_and_dataframe(path, train_csv, val_csv):
    # Load the CSV files
    train_df = pd.read_csv(os.path.join(path, train_csv))
    val_df = pd.read_csv(os.path.join(path, val_csv))
    
    # Encode the categorical labels
    age_encoder = LabelEncoder()
    gender_encoder = LabelEncoder()
    race_encoder = LabelEncoder()

    train_df['age'] = age_encoder.fit_transform(train_df['age'])
    train_df['gender'] = gender_encoder.fit_transform(train_df['gender'])
    train_df['race'] = race_encoder.fit_transform(train_df['race'])

    val_df['age'] = age_encoder.transform(val_df['age'])
    val_df['gender'] = gender_encoder.transform(val_df['gender'])
    val_df['race'] = race_encoder.transform(val_df['race'])
    
    return train_df, val_df, age_encoder, gender_encoder, race_encoder

In [4]:
# for all data
def load_data(path, input_shape):
    train_datagen = ImageDataGenerator(rescale=1./255)
    val_datagen = ImageDataGenerator(rescale=1./255)

    train_gen = train_datagen.flow_from_directory(
        os.path.join(path, 'train'),
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical'
    )

    val_gen = val_datagen.flow_from_directory(
        os.path.join(path, 'val'),
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical'
    )

    return train_gen, val_gen


In [5]:
def load_images(df, path, input_shape):
    images = []
    for _, row in df.iterrows():
        img_path = os.path.join(path, row['file'])
        img = load_img(img_path, target_size=input_shape[:2])
        img_arr = img_to_array(img) / 255.0
        images.append(img_arr)
    return np.array(images)

def load_sample_data(path, input_shape, train_csv, val_csv, sample_percent=0.1):
    # Load the dataframes and label encoders
    train_df, val_df, age_encoder, gender_encoder, race_encoder = load_labels_and_dataframe(path, 
                                                                                            "fairface_label_"+train_csv, 
                                                                                            "fairface_label_"+val_csv)

    # Take a fraction of the dataset
    train_df = train_df.sample(frac=sample_percent)
    val_df = val_df.sample(frac=sample_percent)

    # Load the images and labels
    train_images = load_images(train_df, path, input_shape)
    train_labels = train_df[["age", "gender", "race"]].values
    val_images = load_images(val_df, path, input_shape)
    val_labels = val_df[["age", "gender", "race"]].values

    return train_images, train_labels, val_images, val_labels, age_encoder, gender_encoder, race_encoder

# Autoencoder and Decoder

In [6]:
def create_autoencoder(input_shape):
    input_img = Input(shape=input_shape)

    # Encoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    return autoencoder

In [7]:
def extract_encoder(autoencoder):
    # Extract the encoder layers from the autoencoder model
    input_img = autoencoder.input
    # Assuming the encoder layers are the first 5 layers
    encoded = autoencoder.layers[5].output

    # Create a new model with the encoder layers
    encoder = Model(input_img, encoded)

    return encoder

# Classification Model

In [8]:
def create_classification_model(encoder, num_race_classes, num_age_classes, num_gender_classes):
    # Use the encoder's output as input for the classification model
    encoded_input = encoder.output

    # Add a Flatten layer to convert the feature maps into a 1D vector
    x = Flatten()(encoded_input)

    # Add Dense layers for classification
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)

    # Add output layers for each attribute: race, age, and gender
    race_output = Dense(num_race_classes, activation='softmax', name='race_output')(x)
    age_output = Dense(num_age_classes, activation='softmax', name='age_output')(x)
    gender_output = Dense(num_gender_classes, activation='softmax', name='gender_output')(x)

    # Create the classification model
    classification_model = Model(encoder.input, [race_output, age_output, gender_output])

    return classification_model

## Random Forest Model for Benchmarking

In [9]:
def predict_in_batches(encoder, data, batch_size):
    num_batches = int(np.ceil(len(data) / batch_size))
    predictions = []

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(data))
        batch_data = data[start_idx:end_idx]
        batch_predictions = encoder.predict(batch_data)
        predictions.append(batch_predictions)

    return np.concatenate(predictions, axis=0)

In [10]:
def run_model(data_path, sample_percent=0.1, batch_size=8, epochs = 2):
    input_shape = (224, 224, 3)

    # Load the data
    train_images, train_labels, val_images, val_labels, age_encoder, gender_encoder, race_encoder = load_sample_data(
        data_path, input_shape, "train.csv", "val.csv", sample_percent=sample_percent
    )

    # Create the autoencoder model
    autoencoder = create_autoencoder(input_shape)
    autoencoder.compile(optimizer='adam', loss='mse')

    # Train the autoencoder
    autoencoder.fit(
        train_images, train_images,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(val_images, val_images)
    )

    # Extract the encoder
    encoder = extract_encoder(autoencoder)

    # Encode the train and validation images
    train_embeddings = predict_in_batches(encoder, train_images, batch_size)
    val_embeddings = predict_in_batches(encoder, val_images, batch_size)


    # Flatten the embeddings
    train_embeddings = train_embeddings.reshape(train_embeddings.shape[0], -1)
    val_embeddings = val_embeddings.reshape(val_embeddings.shape[0], -1)

    # Train a classifier on the embeddings
    clf = RandomForestClassifier()
    clf.fit(train_embeddings, train_labels)

    # Predict the labels for the validation set
    val_predictions = clf.predict(val_embeddings)

    # Compute the accuracy for each attribute
    print("Validation accuracy:")
    age_metrics = classification_report(val_labels[:, 0], val_predictions[:, 0], target_names=age_encoder.classes_, zero_division=1)
    gender_metrics = classification_report(val_labels[:, 1], val_predictions[:, 1], target_names=gender_encoder.classes_, zero_division=1)
    race_metrics = classification_report(val_labels[:, 2], val_predictions[:, 2], target_names=race_encoder.classes_, zero_division=1)
    print(f"Age: {age_metrics}")
    print(f"Gender: {gender_metrics}")
    print(f"Race: {race_metrics}")

    return {'age_accuracy': age_metrics,
            'gender_metrics': gender_metrics,
            'race_metrics': race_metrics}

In [11]:
%%capture cap --no-stderr
data_path = "./fairface-img-margin025-trainval/fairface-img-margin025-trainval/"
rf_metrics = run_model(data_path, sample_percent = 0.1, epochs = 50)

2023-05-06 16:29:36.983641: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 33409 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:51:00.0, compute capability: 8.6
2023-05-06 16:29:46.899523: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8800
2023-05-06 16:29:47.561690: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-05-06 16:29:48.056103: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f3abeef3330 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-06 16:29:48.056128: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA A40, Compute Capability 8.6
2023-05-06 16:29:48.060811: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPR

In [12]:
with open('baseline_rf.txt', 'w') as f:
    f.write(cap.stdout)

In [ ]:
print(rf_metrics['age_accuracy'])
print(rf_metrics['gender_metrics'])
print(rf_metrics['race_metrics'])

NameError: name 'rf_metrics' is not defined

In [18]:
# rf_metrics_10_epochs = rf_metrics

In [22]:
!nvidia-smi

Sat May  6 17:05:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          Off  | 00000000:51:00.0 Off |                    0 |
|  0%   50C    P0    83W / 300W |  27676MiB / 46068MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
del x

In [11]:
# clear GPU memory
tf.compat.v1.reset_default_graph()

## Deep Learning Model

In [12]:
def predict_in_batches_tf(model, data, batch_size):
    num_batches = int(np.ceil(len(data) / float(batch_size)))
    all_predictions = []

    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, len(data))
        batch_data = data[start_idx:end_idx]
        batch_predictions = model.predict(batch_data)
        all_predictions.append(batch_predictions)

    # Combine the predictions from all batches
    combined_predictions = [np.concatenate([batch[i] for batch in all_predictions], axis=0) for i in range(len(all_predictions[0]))]
    return combined_predictions

In [13]:
def run_model(data_path, sample_percent=0.1, epochs = 2, encoder_bs = 4, prediction_batch_size=4):
    input_shape = (224, 224, 3)

    # Load the data
    train_images, train_labels, val_images, val_labels, age_encoder, gender_encoder, race_encoder = load_sample_data(
        data_path, input_shape, "train.csv", "val.csv", sample_percent=sample_percent
    )

    num_race_classes = len(race_encoder.classes_)
    num_age_classes = len(age_encoder.classes_)
    num_gender_classes = len(gender_encoder.classes_)

    # Create the autoencoder model
    autoencoder = create_autoencoder(input_shape)
    autoencoder.compile(optimizer='adam', loss='mse')

    # Train the autoencoder
    autoencoder.fit(
        train_images, train_images,
        epochs=epochs,
        batch_size=encoder_bs,
        validation_data=(val_images, val_images)
    )

    # Extract the encoder
    encoder = extract_encoder(autoencoder)

    num_race_classes = len(race_encoder.classes_)
    num_age_classes = len(age_encoder.classes_)
    num_gender_classes = len(gender_encoder.classes_)

    # Create the classification model
    classification_model = create_classification_model(encoder, num_race_classes, num_age_classes, num_gender_classes)

    # Compile the classification model
    classification_model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss={
            'race_output': SparseCategoricalCrossentropy(),
            'age_output': SparseCategoricalCrossentropy(),
            'gender_output': SparseCategoricalCrossentropy()
        },
        metrics={
            'race_output': SparseCategoricalAccuracy(),
            'age_output': SparseCategoricalAccuracy(),
            'gender_output': SparseCategoricalAccuracy()
        }
    )

    # Train the classification model
    classification_model.fit(
        train_images, [train_labels[:, 2], train_labels[:, 0], train_labels[:, 1]],
        epochs=epochs,
        batch_size=prediction_batch_size,
        validation_data=(val_images, [val_labels[:, 2], val_labels[:, 0], val_labels[:, 1]])
    )

    # Predict the labels for the validation set
    #race_predictions, age_predictions, gender_predictions = classification_model.predict(val_images)

    race_predictions, age_predictions, gender_predictions = predict_in_batches_tf(classification_model, 
                                                                                  val_images, batch_size=prediction_batch_size)


    # Compute the accuracy for each attribute
    race_metrics = classification_report(val_labels[:, 2], race_predictions.argmax(axis=1), target_names=race_encoder.classes_, zero_division=1)
    age_metrics = classification_report(val_labels[:, 0], age_predictions.argmax(axis=1), target_names=age_encoder.classes_, zero_division=1)
    gender_metrics = classification_report(val_labels[:, 1], gender_predictions.argmax(axis=1), target_names=gender_encoder.classes_, zero_division=1)

    print("Validation accuracy:")

    print(f"Age: {age_metrics}")
    print(f"Gender: {gender_metrics}")
    print(f"Race: {race_metrics}")

    return {'age_metrics': age_metrics,
            'gender_metrics': gender_metrics,
            'race_metrics': race_metrics}

In [ ]:
%%capture cap --no-stderr
data_path = "./fairface-img-margin025-trainval/fairface-img-margin025-trainval"
classif_results = run_model(data_path, sample_percent = 0.15, epochs = 50)

2023-05-06 18:24:32.735952: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43482 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:51:00.0, compute capability: 8.6
2023-05-06 18:24:45.618146: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8800
2023-05-06 18:24:46.277658: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-05-06 18:24:46.654330: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f2275284b30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-06 18:24:46.654350: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA A40, Compute Capability 8.6
2023-05-06 18:24:46.658701: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPR

In [ ]:
with open('classif.txt', 'w') as f:
    f.write(cap.stdout)

In [22]:
print(classif_results['age_metrics'])
print(classif_results['gender_metrics'])
print(classif_results['race_metrics'])

              precision    recall  f1-score   support

         0-2       0.40      0.17      0.24        35
       10-19       0.19      0.20      0.20       166
       20-29       0.37      0.41      0.39       496
         3-9       0.45      0.47      0.46       230
       30-39       0.25      0.34      0.29       323
       40-49       0.19      0.12      0.15       207
       50-59       0.23      0.11      0.15       120
       60-69       0.00      0.00      0.00        55
more than 70       0.00      0.00      0.00        11

    accuracy                           0.30      1643
   macro avg       0.23      0.20      0.21      1643
weighted avg       0.29      0.30      0.29      1643

              precision    recall  f1-score   support

      Female       0.69      0.64      0.67       764
        Male       0.71      0.75      0.73       879

    accuracy                           0.70      1643
   macro avg       0.70      0.70      0.70      1643
weighted avg       0.70

In [ ]:
!nvidia-smi

# Code to restrict memory consumption
We kept getting extensive errors thus had to drop this part.

In [10]:
# this function is to use data generators instead of actual images. Hopefully, this will require less
# memory to run the code.
def load_sample_data(path, input_shape, train_csv, val_csv, sample_percent=0.1):
    # Load the dataframes and label encoders
    train_df, val_df, age_encoder, gender_encoder, race_encoder = load_labels_and_dataframe(path, 
                                                                                            "fairface_label_"+train_csv, 
                                                                                            "fairface_label_"+val_csv)

    # Take a fraction of the dataset
    train_df = train_df.sample(frac=sample_percent)
    val_df = val_df.sample(frac=sample_percent)

    return train_df, val_df, age_encoder, gender_encoder, race_encoder

In [11]:
def data_generator(df, path, input_shape, batch_size):
    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i+batch_size]
        batch_images = load_images(batch_df, path, input_shape)
        batch_labels = batch_df[["age", "gender", "race"]].values
        yield batch_images, [batch_labels[:, 2], batch_labels[:, 0], batch_labels[:, 1]]

In [14]:
def autoencoder_data_generator(df, data_path, input_shape, batch_size):
    while True:
        for i in range(0, len(df), batch_size):
            batch_df = df.iloc[i:i+batch_size]
            batch_images = load_images(batch_df, data_path, input_shape)
            yield batch_images, batch_images

In [23]:
from tensorflow.data import Dataset
def run_model(data_path, sample_percent=0.1, epochs=2, encoder_bs=4, prediction_batch_size=4):
    input_shape = (224, 224, 3)

    autoencoder_train_dataset = Dataset.from_generator(
        lambda: autoencoder_data_generator(train_df, data_path, input_shape, encoder_bs),
        output_signature=(
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32)
        )
    )

    autoencoder_val_dataset = Dataset.from_generator(
        lambda: autoencoder_data_generator(val_df, data_path, input_shape, encoder_bs),
        output_signature=(
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32)
        )
    )

    # Load the data
    
    train_df, val_df, age_encoder, gender_encoder, race_encoder = load_sample_data(
        data_path, input_shape, "train.csv", "val.csv", sample_percent=sample_percent
    )

    num_race_classes = len(race_encoder.classes_)
    num_age_classes = len(age_encoder.classes_)
    num_gender_classes = len(gender_encoder.classes_)

    
    # Create the autoencoder model
    autoencoder = create_autoencoder(input_shape)
    autoencoder.compile(optimizer='adam', loss='mse')

    # Train the autoencoder
    autoencoder.fit(
        autoencoder_train_dataset,
        epochs=2,
        steps_per_epoch=len(train_df) // encoder_bs,
        validation_data=autoencoder_val_dataset,
        validation_steps=len(val_df) // encoder_bs
    )
    # Extract the encoder
    encoder = extract_encoder(autoencoder)

    # Create the classification model
    classification_model = create_classification_model(encoder, num_race_classes, num_age_classes, num_gender_classes)

    # Compile the classification model
    classification_model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss={
            'race_output': SparseCategoricalCrossentropy(),
            'age_output': SparseCategoricalCrossentropy(),
            'gender_output': SparseCategoricalCrossentropy()
        },
        metrics={
            'race_output': SparseCategoricalAccuracy(),
            'age_output': SparseCategoricalAccuracy(),
            'gender_output': SparseCategoricalAccuracy()
        }
    )

    train_dataset = Dataset.from_generator(
        lambda: data_generator(train_df, data_path, input_shape, prediction_batch_size),
        output_signature=(
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
            (
                tf.TensorSpec(shape=(None,), dtype=tf.int32),
                tf.TensorSpec(shape=(None,), dtype=tf.int32),
                tf.TensorSpec(shape=(None,), dtype=tf.int32)
            )
        )
    )

    val_dataset = Dataset.from_generator(
        lambda: data_generator(val_df, data_path, input_shape, prediction_batch_size),
        output_signature=(
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
            (
                tf.TensorSpec(shape=(None,), dtype=tf.int32),
                tf.TensorSpec(shape=(None,), dtype=tf.int32),
                tf.TensorSpec(shape=(None,), dtype=tf.int32)
            )
        )
    )


    # Train the classification model
    classification_model.fit(
        train_dataset,
        epochs=epochs,
        steps_per_epoch=len(train_df) // prediction_batch_size,
        validation_data=val_dataset,
        validation_steps=len(val_df) // prediction_batch_size
    )

    # Predict the labels for the validation set
    race_predictions, age_predictions, gender_predictions = predict_in_batches_tf(classification_model, 
                                                                                  val_images, batch_size=prediction_batch_size)

    # Compute the accuracy for each attribute
    print("Validation accuracy:")
    print("Race:")
    print(classification_report(val_labels[:, 2], race_predictions.argmax(axis=1), target_names=race_encoder.classes_, zero_division=1))
    print("Age:")
    print(classification_report(val_labels[:, 0], age_predictions.argmax(axis=1), target_names=age_encoder.classes_, zero_division=1))
    print("Gender:")
    print(classification_report(val_labels[:, 1], gender_predictions.argmax(axis=1), target_names=gender_encoder.classes_, zero_division=1))
    
    return encoder

In [24]:
data_path = "./fairface-img-margin025-trainval/fairface-img-margin025-trainval"
x = run_model(data_path, sample_percent = 0.2)

Epoch 1/2


2023-05-06 15:53:24.396061: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


4335/4337 [============================>.] - ETA: 0s - loss: 0.0016

2023-05-06 15:54:08.375280: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


4337/4337 [==============================] - 46s 10ms/step - loss: 0.0016 - val_loss: 6.8946e-04
Epoch 2/2
4337/4337 [==============================] - 45s 10ms/step - loss: 7.2530e-04 - val_loss: 6.1968e-04
Epoch 1/2


2023-05-06 15:54:55.862036: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-06 15:54:57.071306: W tensorflow/core/framework/op_kernel.cc:1818] INVALID_ARGUMENT: TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was (tf.float32, (tf.int32, tf.int32, tf.int32)), but the yielded element was (array([[[[0.47058824, 0.43529412, 0.40784314],
         [0.41568628, 0.38039216, 0.3529412 ],
         [0.39607844, 0.36862746, 0.3372549 ],
         ...,
         [0.89411765, 0.92941177, 0.5647059 ],
         [0.8980392 , 0.92941177, 0.5411765 ],
         [0.8901961 , 0.92156863, 0.5254902 ]],

        [[0.45490196, 0.41960785, 0.39215687],
         [0.41960785, 0.38431373, 0.35686275],
  

InvalidArgumentError: Graph execution error:

2 root error(s) found.
  (0) INVALID_ARGUMENT:  TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was (tf.float32, (tf.int32, tf.int32, tf.int32)), but the yielded element was (array([[[[0.47058824, 0.43529412, 0.40784314],
         [0.41568628, 0.38039216, 0.3529412 ],
         [0.39607844, 0.36862746, 0.3372549 ],
         ...,
         [0.89411765, 0.92941177, 0.5647059 ],
         [0.8980392 , 0.92941177, 0.5411765 ],
         [0.8901961 , 0.92156863, 0.5254902 ]],

        [[0.45490196, 0.41960785, 0.39215687],
         [0.41960785, 0.38431373, 0.35686275],
         [0.40392157, 0.3764706 , 0.34509805],
         ...,
         [0.9137255 , 0.9372549 , 0.5921569 ],
         [0.9019608 , 0.92941177, 0.5647059 ],
         [0.8901961 , 0.91764706, 0.54509807]],

        [[0.4117647 , 0.3764706 , 0.34901962],
         [0.40392157, 0.36862746, 0.34117648],
         [0.40784314, 0.38039

    [0.1882353 , 0.20392157, 0.2       ],
         [0.19215687, 0.20784314, 0.20392157],
         [0.19215687, 0.20784314, 0.20392157]],

        [[0.56078434, 0.56078434, 0.5529412 ],
         [0.5294118 , 0.5294118 , 0.52156866],
         [0.4745098 , 0.4745098 , 0.46666667],
         ...,
         [0.19215687, 0.20784314, 0.20392157],
         [0.19607843, 0.21176471, 0.20784314],
         [0.19607843, 0.21176471, 0.20784314]],

        [[0.57254905, 0.57254905, 0.5647059 ],
         [0.5411765 , 0.5411765 , 0.53333336],
         [0.48235294, 0.48235294, 0.4745098 ],
         ...,
         [0.19607843, 0.21176471, 0.20784314],
         [0.2       , 0.21568628, 0.21176471],
         [0.2       , 0.21568628, 0.21176471]]],


       [[[0.12941177, 0.13725491, 0.11764706],
         [0.12941177, 0.13725491, 0.11764706],
         [0.12941177, 0.13725491, 0.11764706],
         ...,
         [0.7882353 , 0.6431373 , 0.5411765 ],
         [0.78039217, 0.63529414, 0.53333336],
         [0.780